In [1]:
import numpy as np

In [2]:
num_bread = 1000

In [4]:
# Making imaginary bread.

# Weights: 50 grams, with a standard deviation of 3 grams.
weights = 50 + 3*np.random.randn(num_bread, 1)

# Temperature: 22 degrees, with a standard deviation of 5 degrees.
temps = 22 + 5*np.random.randn(num_bread, 1)

# Volume in ml: 3ml per gram of weight, plus 4ml per degree above 22, 
# plus some noise.
volume = 3*weights + 4*(temps - 22) + 5*np.random.randn(num_bread, 1)

In [5]:
X = np.c_[weights, temps]
y = volume

In [6]:
X.shape, y.shape

((1000, 2), (1000, 1))

In [11]:
# Splitting the data into training and test sets.
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
)

In [12]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

# Here is where the model learns its optimal parameters.
model.fit(X_train, y_train)

LinearRegression()

In [14]:
model.intercept_, model.coef_

(array([-87.71039787]), array([[2.9642998 , 4.05972089]]))

O modelo final é:

$$
\hat{y} = \overset{\theta_0}{-87.71} + \overset{\theta_1}{2.96} x_1 + \overset{\theta_2}{4.06} x_2
$$

In [18]:
y_pred = model.predict(X_test)

In [17]:
from sklearn.metrics import root_mean_squared_error

root_mean_squared_error(y_test, y_pred)

np.float64(5.093186243609687)

Equivalently, I could use the *normal equation* for fitting: 

In [22]:
X_train_augmented = np.c_[np.ones((X_train.shape[0], 1)), X_train]

In [25]:
theta_opt = np.linalg.inv(X_train_augmented.T @ X_train_augmented) @ X_train_augmented.T @ y_train

In [26]:
theta_opt

array([[-87.71039787],
       [  2.9642998 ],
       [  4.05972089]])

In [27]:
model.intercept_, model.coef_

(array([-87.71039787]), array([[2.9642998 , 4.05972089]]))

Suppose the baker made a mistake and duplicated a data column:

In [28]:
X = np.c_[weights, weights, temps]
y = volume

In [29]:
X[:5,:]

array([[45.57719403, 45.57719403, 15.07321174],
       [49.67665931, 49.67665931, 23.45300092],
       [54.15154476, 54.15154476, 19.4866593 ],
       [46.7096675 , 46.7096675 , 20.8072701 ],
       [53.96506039, 53.96506039,  7.87149008]])

In [30]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
)

In [31]:
model = LinearRegression()

model.fit(X_train, y_train)

LinearRegression()

In [32]:
model.intercept_, model.coef_

(array([-87.71039787]), array([[1.4821499 , 1.4821499 , 4.05972089]]))

In [34]:
X_train_augmented = np.c_[np.ones((X_train.shape[0], 1)), X_train]

In [35]:
np.linalg.inv(X_train_augmented.T @ X_train_augmented) @ X_train_augmented.T @ y_train

LinAlgError: Singular matrix

In [38]:
M = X_train_augmented.T @ X_train_augmented

In [39]:
np.linalg.det(M)

np.float64(0.0)

In [40]:
U, S, Vt = np.linalg.svd(X_train_augmented)

In [42]:
X_train_augmented.shape

(800, 4)

In [41]:
U.shape, S.shape, Vt.shape

((800, 800), (4,), (4, 4))

Another example

In [47]:
temps_F = 1.8*temps + 32

In [48]:
X = np.c_[weights, temps, temps_F]

In [49]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
)

In [50]:
X_train_augmented = np.c_[np.ones((X_train.shape[0], 1)), X_train]

In [51]:
X_train_augmented[:5,:]

array([[ 1.        , 57.16841295, 35.53286924, 95.95916464],
       [ 1.        , 49.99225374, 16.05982378, 60.90768281],
       [ 1.        , 47.85025358, 19.27330812, 66.69195462],
       [ 1.        , 50.53960314, 27.15957096, 80.88722773],
       [ 1.        , 48.87767233, 25.1945085 , 77.3501153 ]])

In [81]:
alpha = 1e-3 / X_train_augmented.shape[1]
M = X_train_augmented.T @ X_train_augmented + alpha*np.eye(X_train_augmented.shape[1])

In [82]:
(np.linalg.inv(M) @ M).round(2)

array([[ 1., -0., -0., -0.],
       [ 0.,  1.,  0.,  0.],
       [-0., -0.,  1., -0.],
       [ 0.,  0.,  0.,  1.]])

In [83]:
U, S, Vt = np.linalg.svd(X_train_augmented)

In [84]:
theta = np.linalg.inv(X_train_augmented.T @ X_train_augmented \
    + alpha * np.eye(X_train_augmented.shape[1])) @ X_train_augmented.T @ y_train

In [85]:
theta

array([[-0.58909635],
       [ 2.96429855],
       [ 8.96028986],
       [-2.7225385 ]])

In [86]:
model = LinearRegression()

model.fit(X_train, y_train)

LinearRegression()

In [87]:
model.intercept_, model.coef_

(array([-1.90306764e+13]),
 array([[ 2.96429980e+00, -1.07047555e+12,  5.94708636e+11]]))

In [91]:
from sklearn.linear_model import Ridge

model = Ridge(alpha=1e-3)

model.fit(X_train, y_train)

model.intercept_, model.coef_

(array([-142.86129955]), array([[2.96429935, 0.95748125, 1.72346643]]))